In [2]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [3]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
def get_quarter(selldate):
    month_d = selldate.month
    if month_d in [1, 2, 3]: quarter = 1
    if month_d in [4, 5, 6]: quarter = 2
    if month_d in [7, 8, 9]: quarter = 3
    if month_d in [10, 11, 12]: quarter = 4
    return quarter
melb_df['Quarter'] = melb_df['Date'].apply(get_quarter)
display(melb_df['Quarter'].value_counts())

3    4873
2    4359
4    2329
1    2019
Name: Quarter, dtype: int64

In [26]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

In [51]:
display(melb_df.sort_values(by='AreaRatio', ignore_index=True, ascending=False).loc[1558, 'BuildingArea'])


126.0

In [56]:
display(melb_df[(melb_df['Type'] == 'townhouse') & (melb_df['Rooms'] > 2)].sort_values(
    by=['Rooms', 'MeanRoomsSquare'], ignore_index=True, ascending=[True, False]
    ).loc[18, 'Price'])

1300000.0

In [61]:
melb_df.groupby('Rooms')['Price'].mean().sort_values()

Rooms
1     4.338245e+05
2     7.750812e+05
10    9.000000e+05
3     1.076081e+06
4     1.445282e+06
8     1.602750e+06
6     1.849366e+06
5     1.870260e+06
7     1.920700e+06
Name: Price, dtype: float64

In [62]:
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

Regionname
Western Victoria              0.011579
Southern Metropolitan         0.043080
Eastern Metropolitan          0.047890
Northern Metropolitan         0.049639
Western Metropolitan          0.051251
South-Eastern Metropolitan    0.073411
Northern Victoria             0.084455
Eastern Victoria              0.147067
Name: Lattitude, dtype: float64

In [66]:
import datetime as dt
melb_df[(melb_df['Date'] <= dt.datetime(2017, 9, 1)) & (melb_df['Date'] >= dt.datetime(2017, 5, 1))].groupby('SellerG')['Price'].sum().sort_values()

SellerG
LITTLE             2742000.0
Cayzer             4439000.0
Burnham            4550500.0
Moonee             7328000.0
Thomson            8332000.0
Bells              8656000.0
Alexkarbon        10985000.0
McDonald          14637500.0
Rendina           15422276.0
Nick              16890000.0
Douglas           18341000.0
Buckingham        19033000.0
C21               19515000.0
Eview             19791500.0
Collins           20217000.0
Philip            22051800.0
Chisholm          23225000.0
Williams          23297000.0
Love              23365500.0
Purplebricks      23401000.0
O'Brien           23855508.0
HAR               25568000.0
Village           26473000.0
RW                29261000.0
Raine             30687700.0
Stockdale         35409800.0
Sweeney           36882750.0
Gary              39138400.0
Hodges            43231000.0
YPA               46354350.0
Miles             47582000.0
Kay               48569500.0
RT                50498000.0
Brad              55955000.0
Jas   

In [69]:
melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median'
)

Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126.0,126.0,141.0,177.0,126.0,216.5,126.0,126.0
townhouse,88.0,114.0,126.0,159.5,152.0,NaN,NaN,NaN,NaN
unit,69.5,110.0,126.0,126.0,NaN,171.0,NaN,126.0,NaN


In [74]:
melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
).round().sort_values(by='unit', ascending=False)

Type,house,townhouse,unit
SellerG,,,
Nick,2111984.0,780000.0,993643.0
Kay,2471453.0,1648818.0,851661.0
Cayzer,1608258.0,1277333.0,832333.0
Marshall,2161028.0,1590250.0,770045.0
Philip,1068569.0,737000.0,706256.0
RT,1841567.0,1261643.0,703510.0
C21,969340.0,768750.0,700700.0
Fletchers,1480143.0,1248796.0,691625.0
Williams,1258358.0,858714.0,685386.0
